In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from scipy.stats import pearsonr, spearmanr, ttest_ind, ttest_rel

# 作业要求
该数据采集自THUIR用户实验室。场景是用户在移动环境下（安卓应用）阅读新闻信息流。实验过程中会记录用户的行为并让用户进行一系列的标注。

用户交互界面和一般的新闻信息流App类似（如今日头条），是一个单栏的新闻列表，列表中每个元素是一条新闻，包括新闻标题和三张图片。
用户可以向下滑动看完整个列表。当点击某个元素时，界面会跳转到完整新闻内容页面。

实验用户（被试）除了预实验之外，正式实验会被要求做11个阅读任务（Task），每个任务是一个新闻列表，包含15条不同的新闻，且不同任务列表间的新闻无交集。
每个用户的11个阅读列表原始是相同的，但列表顺序会打乱，且可能会在每个任务列表中插入一些低质量的新闻。


被试会在给定的1280\*720像素手机上进行实验。  
（Stage 0）在实验开始时，会询问用户一些问题。比如会让用户标注他在5个主题上的偏好，1-5分递增。（社会social, 娱乐enterprise, 科技technology, 历史history, 体育sport）  
（Stage 1）为了让被试熟悉实验环境，会进行一项预实验，不记录。接下来开始正式实验。  
（Stage 2）被试阅读任务列表的场景信息和注意事项。  
（Stage 3）被试阅读新闻列表，不一定每条都读，按自己平时的阅读兴趣来阅读，也不限时间，可以随时退出列表。  
（Stage 4）结束该任务列表后，用户会被要求对新闻列表的整体情况给一些包括满意度在内的反馈。  
（Stage 5）在给出列表反馈后，用户也会被要求给出列表中每条新闻的偏好反馈。  
Stage 2-5会重复直到用户完成所有11个任务列表。  
在实验结束后，会再次询问被试一些问题，比如实验后在5个主题上的偏好，以对比实验前。  

为了探究用户在不同阶段对新闻的偏好：
  - Before-Read：在用户点击新闻后跳转到内容页面前会询请该用户对新闻进行偏好评分，1-5递增。
  - After-Read：在用户阅读完新闻，离开内容页面时，会再次询问用户对新闻对偏好，并对内容质量和标题内容一致性进行评分，1-5递增。
  - Post-Task：上面两种偏好可能受到列表内容、新闻位置等影响，为消除这些影响，在任务完成之后，会请用户对列表中所有的新闻再次进行偏好评分（包括未点击的新闻）

为了探究用户点击新闻的原因：
在After-Read阶段，会询问用户点击该新闻的原因，分5种  
  1. 被标题吸引  
  2. 被图片吸引  
  3. 被话题吸引  
  4. 误点击  
  5. 以上都不是  

为了探究新闻质量对用户偏好对影响：
会在每个任务列表中插入一些低质量的新闻，插入数目（condition）有四种情况，0/3/6/9个。

# News Dataframe

In [2]:
data_dir = "./hw1-data/"

In [3]:
df_news = pd.read_pickle(os.path.join(data_dir, "df_news.pkl"))

print("User Num:", len(set(df_news["userid"])))
print("News Num:", len(df_news))
print("Clicked News Num:", len(df_news[df_news['ifclick']]))

User Num: 26
News Num: 4290
Clicked News Num: 1337


|键值|解释|
|:--|--:|
|userid|用户在数据集中的编号|
|newsid|新闻在数据集中的编号|
|topic|新闻的主题|
|topic_pref|实验开始时，用户标注的对该主题的偏好|
|post_topic_pref|所有实验任务结束后，用户标注的对该主题的偏好|
|taskid|任务列表在数据集中的编号|
|task_pos|该任务列表是该用户实验的11个任务列表中的第几个|
|condition|该任务列表中插入的低质量新闻的数目|
|equality|由专家标注的新闻质量|
|imp_position|新闻在当前任务列表中的位置|
|ifview|用户是否在列表中刷到了该新闻|
|ifclick|用户是否点击了该新闻|
|viewport_time|列表页面）用户停留在该新闻上的时长（ms）|
|click_position|用户在该新闻任务列表中点击的新闻位置（按时间顺序）|
|task_clk_num|在该任务列表中用户点击的新闻总数|
|read_depth|用户阅读内容的像素深度（pixel）|
|read_ratio|用户阅读内容的占该新闻所有内容的比例|
|read_speed|用户阅读新闻的平均速度（pixel/s）|
|max_scroll_speed|用户阅读新闻时的最大滚动速度（pixel/s）|
|max_scroll_interval|用户阅读新闻时两次滑动间的最大间隔（ms）|
|direction_change_times|用户在阅读时滑动方向改变的次数|
|dwell_time|（内容页面）用户阅读该新闻的时长（s）|
|pre_pref|点击后、阅读前用户对该新闻对偏好评分|
|read_pref|阅读完后用户对该新闻对偏好评分|
|whyclick|用户所给的点击该新闻的原因 （1.被标题吸引 2.被图片吸引 3.被话题吸引 4.误点击 5.以上都不是）|
|uquality|用户对该新闻的质量评分|
|utitle|用户对该新闻标题和内容一致性的评分|
|post_pref|用户在完成当前任务列表后对该新闻的偏好评分|


In [5]:
df_news.head()

,userid,newsid,topic,topic_pref,post_topic_pref,taskid,task_pos,condition,equality,imp_position,...,max_scroll_speed,max_scroll_interval,direction_change_times,dwell_time,pre_pref,read_pref,whyclick,uquality,utitle,post_pref
0,150406126,15993932,历史,2,1,8,0,c3,0,1,...,12.75,2914.0,12.0,34.0,3.0,2.0,1,2.0,3.0,2.0
1,150406126,21226182,科技,2,3,8,0,c3,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,3.0
2,150406126,20993528,历史,2,1,8,0,c3,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,1.0
3,150406126,21169773,历史,2,1,8,0,c3,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,2.0
4,150406126,20730284,社会,4,4,8,0,c3,1,5,...,3.75,11971.0,10.0,82.0,4.0,4.0,"1,3",4.0,5.0,3.0


# Task Dataframe

In [6]:
df_task = pd.read_pickle(os.path.join(data_dir, 'df_task.pkl'))

print("Task Num:", len(df_task))

Task Num: 286


|键值|释义|
|:--|:--|
|user_id|用户在数据集中的编号|
|task_id|任务列表在数据集中的编号|
|topic|任务列表中包含的新闻主题|
|task_pos|该任务列表是该用户实验的11个任务列表中的第几个|
|condition|该任务列表中插入的低质量新闻的数目|
|satisfaction|用户对该任务列表的满意度|
|sat_zscore|用户对该任务列表的满意度（标准化去除用户打分倾向）|
|quality|无用列，请忽略|
|num_low_news_imp|同condition，该任务列表中的低质量新闻数目|
|num_high(low)_news_clk|该任务列表中点击的高（低）质量新闻数目|
|view_cnt|在任务列表中用户有刷到的新闻数|
|click_cnt|该任务列表中用户点击的新闻数|
|ctr|该任务列表的用户点击率|
|first_click_pos|该任务列表中用户第一次点击的新闻在列表中的位置|
|click_pos_list|该任务列表中用户所有点击的新闻的位置|
|clk_dwells|该任务列表中用户所有点击新闻所阅读的时间（s）（None表示没有阅读，可能是误点击）|
|browse_time|用户浏览该列表所用的时间（ms）|
|post_prefs| 该列表任务完成后对列表里所有新闻的偏好标注|
|pre_prefs| 对于点击且阅读的新闻，点击标题后阅读内容前，用户给出的对新闻的偏好评分（按click_pos_list去重）|
|read_prefs| 对于点击且阅读的新闻，阅读完成后给出的对新闻偏好的评分（按click_pos_list去重）|
|<sum,max,min>_post_prefs| 该列表任务完成后对列表里所有新闻的偏好标注的均值（和、最大值、最小值）|
|<sum,max,min>_click_post_prefs| 该列表任务完成后对列表里点击过的新闻的偏好标注的均值（和、最大值、最小值）|
|<sum,max,min>_pre_prefs| 对于点击的新闻，点击标题后阅读内容前，用户给出的对新闻的偏好评分的均值（和、最大值、最小值）|
|<sum,max,min>_read_prefs| 对于点击的新闻，阅读完成后给出的对新闻偏好的评分的均值（和、最大值、最小值）|
|last_pre_prefs| 最后一个点击新闻，在点击后且阅读内容前给出的偏好评分|
|last_read_prefs| 最后一个点击新闻，阅读完成后给出的对新闻偏好的评分|
|last_post_prefs| 最后一个点击新闻，该列表任务完成后对新闻对偏好评分|

In [9]:
df_task.head()

,userid,task_id,topic,task_pos,condition,satisfaction,sat_zscore,quality,num_low_news_imp,num_high_news_clk,...,sum_pre_prefs,max_pre_prefs,min_pre_prefs,mean_read_prefs,sum_read_prefs,max_read_prefs,min_read_prefs,last_pre_prefs,last_read_prefs,last_post_prefs
0,150406126,8,mixed,0,c3,5,-0.246269,5,3,5,...,20.0,5.0,3.0,3.60,18.0,5.0,2.0,4.0,4.0,1
1,150406126,7,mixed,1,c9,5,-0.246269,5,9,1,...,25.0,5.0,4.0,4.00,24.0,4.0,4.0,4.0,4.0,2
2,150406126,9,mixed,2,c0,6,0.656716,6,0,4,...,17.0,5.0,4.0,4.25,17.0,5.0,4.0,4.0,4.0,1
3,150406126,6,mixed,3,c6,6,0.656716,5,6,3,...,21.0,5.0,4.0,3.80,19.0,4.0,3.0,4.0,4.0,1
4,150406126,4,社会,4,c0,5,-0.246269,5,0,4,...,16.0,4.0,4.0,3.25,13.0,4.0,2.0,4.0,4.0,3


# User Dataframe

In [10]:
df_user = pd.read_pickle(os.path.join(data_dir, 'df_user.pkl'))

print("User Num:", len(df_user))

User Num: 26


|键值|释义|
|:--|:--|
|userid| user ID 用户在数据集中的编号|
|age| user age 用户年龄|
|exp_id, exp_news, exp_newsrec| useless，无用列，请忽略|
|pre_topic_prefs| [(topic, rating)] 用户在实验前对每个主题的偏好打分|
|post_topic_prefs| [(topic, rating)] 用户在实验后对每个主题的偏好打分|
|tasks| [(task_id, condition, satisfaction, quality, None)] [(任务编号，任务的低质量新闻数，用户对任务列表满意度，用户对任务质量的标注，useless无用列)]|
|satisfaction| [satisfaction] 用户对每个任务列表的满意度|
|post_prefs_mean(var)| mean(var)([post-task preference]) 用户在任务列表完成后对所有列表所有新闻的偏好评分的均值（方差）|
|pre_prefs_mean(var)| mean(var)([pre-task preference]) 用户在点击后阅读前对新闻的偏好评分的均值（方差）|
|read_prefs_mean(var)| mean(var)([read-task preference]) 用户在阅读后对新闻的偏好评分的均值（方差）|

In [12]:
df_user.head()

,userid,age,exp_id,exp_news,exp_newsrec,pre_topic_prefs,post_topic_prefs,tasks,satisfactions,post_prefs_mean,post_prefs_var,pre_prefs_mean,pre_prefs_var,read_prefs_mean,read_prefs_var
0,150406126,21,0,3,3,"[(社会, 4), (娱乐, 3), (科技, 2), (体育, 2), (历史, 2)]","[(社会, 4), (娱乐, 5), (科技, 3), (体育, 1), (历史, 1)]","[(8, c3, 5, 5, None), (7, c9, 5, 5, 2), (9, c0...","[5, 5, 6, 6, 5, 6, 3, 7, 4, 6, 5]",1.878788,1.100459,4.071429,0.102041,3.732143,0.481824
1,2015012721,20,4,3,2,"[(社会, 4), (娱乐, 5), (科技, 3), (体育, 4), (历史, 2)]","[(社会, 2), (娱乐, 5), (科技, 1), (体育, 2), (历史, 4)]","[(8, c0, 3, 4, None), (7, c3, 5, 5, 2), (9, c6...","[3, 5, 6, 2, 2, 5, 5, 5, 5, 2, 6]",2.333333,1.701010,3.408163,0.404831,3.265306,1.092878
2,2015310479,26,7,3,3,"[(社会, 2), (娱乐, 4), (科技, 3), (体育, 3), (历史, 5)]","[(社会, 2), (娱乐, 4), (科技, 3), (体育, 3), (历史, 5)]","[(6, c9, 3, 3, None), (8, c0, 5, 5, 2), (7, c3...","[3, 5, 3, 3, 6, 4, 4, 6, 3, 7, 4]",3.387879,1.679853,3.636364,1.201102,3.222222,1.667789
3,2015310968,24,3,4,1,"[(社会, 5), (娱乐, 5), (科技, 4), (体育, 1), (历史, 1)]","[(社会, 3), (娱乐, 5), (科技, 5), (体育, 1), (历史, 2)]","[(6, c6, 3, 4, None), (8, c3, 4, 5, 2), (7, c9...","[3, 4, 4, 6, 6, 4, 5, 5, 2, 5, 2]",2.412121,1.624096,4.229167,0.426649,4.145833,0.666233
4,2016011697,19,1,3,1,"[(社会, 4), (娱乐, 1), (科技, 4), (体育, 2), (历史, 3)]","[(社会, 3), (娱乐, 2), (科技, 5), (体育, 1), (历史, 4)]","[(7, c9, 5, 5, None), (9, c0, 2, 3, 1), (6, c6...","[5, 2, 6, 5, 5, 2, 5, 4, 4, 5, 6]",2.642424,1.538806,3.975610,0.365259,3.658537,1.078525
